In [5]:
import rpy2.robjects as robjects
import pandas as pd
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

robjects.r("source('lib/DataGathering.r')")

make_acs_table_t_r = robjects.globalenv["make_acs_table_t"]
make_acs_table_bg_r = robjects.globalenv["make_acs_table_bg"]

# Convert the R DataFrame to a pandas DataFrame
with localconverter(robjects.default_converter + pandas2ri.converter):
    make_acs_table_t = robjects.conversion.rpy2py(make_acs_table_t_r)
    make_acs_table_bg = robjects.conversion.rpy2py(make_acs_table_bg_r)

R[write to console]: To install your API key for use in future sessions, run this function with `install = TRUE`.



In [6]:
acs_table_t_r = make_acs_table_t(2020)
acs_table_bg_r = make_acs_table_bg(2020)

R[write to console]: Getting data from the 2016-2020 5-year ACS

R[write to console]: Downloading feature geometry from the Census website.  To cache shapefiles for use in future sessions, set `options(tigris_use_cache = TRUE)`.

R[write to console]: Using FIPS code '37' for state 'NC'

R[write to console]: Using FIPS code '063' for 'Durham County'

R[write to console]: Using FIPS code '37' for state 'NC'

R[write to console]: Using FIPS code '063' for 'Durham County'

R[write to console]: Using FIPS code '37' for state 'NC'

R[write to console]: Using FIPS code '063' for 'Durham County'

R[write to console]: Using FIPS code '37' for state 'NC'

R[write to console]: Using FIPS code '063' for 'Durham County'

R[write to console]: Getting data from the 2016-2020 5-year ACS

R[write to console]: Downloading feature geometry from the Census website.  To cache shapefiles for use in future sessions, set `options(tigris_use_cache = TRUE)`.

R[write to console]: Using FIPS code '37' for state 

In [7]:
# Convert the R DataFrame to a pandas DataFrame
with localconverter(robjects.default_converter + pandas2ri.converter):
    acs_table_t = robjects.conversion.rpy2py(acs_table_t_r)
    acs_table_bg = robjects.conversion.rpy2py(acs_table_bg_r)

In [8]:
acs_table_t.head()

,GEOID,NAME,estimate_pop_total,estimate_white,estimate_black,estimate_aian,estimate_asian,estimate_nhpi,estimate_other,estimate_multi,...,pct_rent_over_030,mortgage_over_030,pct_mortgage_over_030,pct_bach_higher,pct_occ_owner,pct_single_parents,pct_non_eng,pct_vacant,pct_owner_occupied,tract_num
1,37063000200,"Census Tract 2, Durham County, North Carolina",3309.0,1610.0,795.0,5.0,104.0,0.0,54.0,137.0,...,0.341894,138.0,0.352041,0.543154,0.454943,0.248855,0.288571,0.137358,0.454943,2
2,37063001710,"Census Tract 17.10, Durham County, North Carolina",4670.0,1238.0,2351.0,0.0,172.0,0.0,0.0,79.0,...,0.431138,77.0,0.107843,0.313495,0.612978,0.653815,0.312200,0.089662,0.612978,17.10
3,37063000500,"Census Tract 5, Durham County, North Carolina",4679.0,1615.0,2189.0,20.0,361.0,0.0,0.0,93.0,...,0.672400,76.0,0.257627,0.422787,0.251838,0.633523,0.234426,0.133298,0.251838,5
4,37063002026,"Census Tract 20.26, Durham County, North Carolina",8623.0,1048.0,5020.0,0.0,128.0,0.0,124.0,135.0,...,0.491273,337.0,0.457880,0.295618,0.344108,0.603747,0.442734,0.090352,0.344108,20.26
5,37063001711,"Census Tract 17.11, Durham County, North Carolina",3994.0,1190.0,1977.0,0.0,69.0,0.0,0.0,135.0,...,0.519796,23.0,0.065341,0.273745,0.221488,0.677419,0.255361,0.054538,0.221488,17.11


In [9]:
from lib.dataloader_alex import get_parcels, get_du_est, add_columns_from_csv